In [1]:
import time

from src.swagger import SwaggerParser
from src.config import Config
import json
import os

os.environ["model"] = "gpt-4o-2024-05-13"
os.environ["language_model_key"] = ""

# root_path = "/Users/naariah/Documents/Python_Codes/api-suts/specifications/v3/languageTool.json"
root_path = "/Users/naariah/Documents/Python_Codes/api-suts/specifications/v3/gitlab-projects-13.json"
config = Config()
config.swagger = root_path
parser = SwaggerParser(config)
operations = parser.extract()

2024-05-23 16:37:31.650 | DEBUG    | src.swagger:__init__:28 - Parsing specification file


In [5]:
operations

[get:/projects,
 post:/projects,
 post:/projects/user/{user_id},
 get:/projects/{id},
 put:/projects/{id},
 delete:/projects/{id},
 get:/projects/{id}/users,
 post:/projects/{id}/fork,
 delete:/projects/{id}/fork,
 get:/projects/{id}/forks,
 post:/projects/{id}/star,
 post:/projects/{id}/unstar,
 get:/projects/{id}/starrers,
 get:/projects/{id}/languages,
 post:/projects/{id}/archive,
 post:/projects/{id}/unarchive,
 post:/projects/{id}/restore,
 post:/projects/{id}/uploads,
 post:/projects/{id}/share,
 delete:/projects/{id}/share/{group_id},
 get:/projects/{id}/hooks,
 post:/projects/{id}/hooks,
 get:/projects/{id}/hooks/{hook_id},
 put:/projects/{id}/hooks/{hook_id},
 delete:/projects/{id}/hooks/{hook_id},
 post:/projects/{id}/housekeeping,
 put:/projects/{id}/transfer,
 post:/projects/{id}/fork/{forked_from_id},
 get:/projects/{id}/snapshot,
 get:/users/{user_id}/projects,
 get:/users/{user_id}/starred_projects]

In [3]:
from src.rest import PathParam

info = []
for operation in operations:
    if operation.__repr__() == "get:/projects/{id}/hooks/{hook_id}":
        for param in operation.parameters:
            if isinstance(param, PathParam):
                info.append(
                    {
                        "name": param.factor.get_global_name, 
                        "description": param.factor.description
                    }
                )

In [4]:
info

[{'name': 'id', 'description': 'The ID or URL-encoded path of the project.'},
 {'name': 'hook_id', 'description': 'The ID of a project hook.'}]

In [ ]:
from openai import OpenAI
from src.template import *
from src.languagemodel.llm import count_tokens

client = OpenAI(api_key=os.environ["language_model_key"])
messages = []
messages.append({"role": "system", "content": SystemRole.SYS_ROLE_SEQUENCE})
prompt = INFO.SEQUENCE.format(info) + Task.SEQUENCE
messages.append({"role": "user", "content": prompt})

In [ ]:
messages

In [ ]:
count_tokens(messages, os.environ["model"])

In [ ]:
from loguru import logger

while True:
    try:
        start = time.time()
        response = client.chat.completions.create(
            model=os.environ["model"],
            messages=messages,
            temperature=0.5,
            top_p=0.99,
            frequency_penalty=0,
            presence_penalty=0,
            max_tokens=4096,
            response_format={"type": "json_object"}
        )
        end = time.time()
        print("Time taken: ", end - start)
        break
    except Exception as e:
        logger.error(f"Error in calling language model: {e}")
        logger.error("Retrying...")

In [ ]:
json.loads(response.choices[0].message.content)